In [37]:
from math import log2 as log
import pandas as pd
import numpy as np
import pprint

In [38]:
df = pd.read_csv('sales.csv')
df

,Age,Income,Gender,MaritialStatus,Buys
0,<21,High,Male,Single,No
1,<21,High,Male,Married,No
2,21-35,High,Male,Single,Yes
3,>35,Medium,Male,Single,Yes
4,>35,Low,Female,Single,Yes
5,>35,Low,Female,Married,No
6,21-35,Low,Female,Married,Yes
7,<21,Medium,Male,Single,No
8,<21,Low,Female,Married,Yes
9,>35,Medium,Female,Single,Yes


In [39]:
n = df.shape[1]
total = df.shape[0]
cols = ['Age','Income','Gender','Marital Status']
print(n)
print(total)

5
14


In [40]:
def information_gain(df):
  Class = df.keys()[-1]
  P_type = df.loc[df[Class]== 'Yes'].count().sum()/n
  N_type = df.loc[df[Class]== 'No'].count().sum()/n
  Info_gain = - (P_type/(P_type+N_type))*(log(P_type/(P_type+N_type))) - (N_type/(N_type+P_type))*(log(N_type/(N_type+P_type)))
  return Info_gain

In [41]:
def find_entropy_attribute(df,col):
    values = df[col].unique()
    Class = df.keys()[-1]
    entropy = 0
    for value in values:
        P_type = df.loc[(df[Class]== 'Yes') & (df[col] == str(value))].count().sum()/n
        N_type = df.loc[(df[Class]== 'No') & (df[col] == str(value))].count().sum()/n
        if(P_type==0 or N_type==0):
            Info_gain = 0
        else:
            Info_gain = - (P_type/(P_type+N_type))*(log(P_type/(P_type+N_type))) - (N_type/(N_type+P_type))*(log(N_type/(N_type+P_type)))
        Prob = (df.loc[df[col] == str(value)].count().sum()/n)/total
        entropy = entropy + Prob*Info_gain
    return entropy  

In [42]:
def find_best(df):
  Gain = {}
  for keys in df.keys()[:-1]:
    Gain[keys]= information_gain(df)-find_entropy_attribute(df,keys)
  return max(Gain, key=Gain.get)

In [43]:
def get_subtable(df, node,value):
  return df[df[node] == value].reset_index(drop=True)

In [44]:
def buildTree(df,tree=None): 
    Class = df.keys()[-1]  
    node = find_best(df)
    attValue = np.unique(df[node])
     
    if tree is None:                    
        tree={}
        tree[node] = {}

    for value in attValue:
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable['Buys'],return_counts=True)                        
        if len(counts)==1:
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = buildTree(subtable) 
                   
    return tree

In [45]:
def predict_outcome(inst,tree):
    for nodes in tree.keys():        
        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0
        if type(tree) is dict:
            prediction = predict(inst, tree)
        else:
            prediction = tree
            break                                
    return prediction

In [46]:
tree = buildTree(df)
pprint.pprint(tree)

{'Age': {'21-35': 'Yes',
         '<21': {'Gender': {'Female': 'Yes', 'Male': 'No'}},
         '>35': {'MaritialStatus': {'Married': 'No', 'Single': 'Yes'}}}}


In [47]:
test = {'Age': '<21','Income' : 'Low', 'Gender': 'Female', 'Marital Status' : 'Married'}
test_data = pd.Series(test)
test_prediction = predict(test_data,tree)
print(test_prediction)

Yes
